# Network Analysis

StrainDesign provides canonical functions for maximizing and minimizing metabolic fluxes in network. The output format is identical to the ones of COBRApy's functions.

## Flux optimization (FBA/pFBA)

Flux balance analysis is a single linear program that optimizes a flux rate under the given steady-state network constraints. For the case of a growth-rate maximization, the problem is written as:
$$\text{maximize}\:\:\: v_{growth} \\ \text{subject to} \\ \mathbf{S\, v = 0} \\ \mathbf{lb \le v \le ub}$$
Where $ \mathbf{S}$ is the stoichiometric matrix, $\mathbf{v}$ is the vector of metabolic flux rates and $\mathbf{lb}$ and $\mathbf{ub}$ are the physiological lower and upper bounds of each flux rate. $\mathbf{S\, v = 0}$ represents all steady-state constraints, and $\mathbf{lb \le v \le ub}$ the flux ranges.

All of the following computation examples will require the COBRApy and the StrainDesign package. Here, we load both packages and the e_coli_core model from Bigg:

In [1]:
import cobra
import straindesign as sd
model = cobra.io.load_model('e_coli_core')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-13


An FBA is launched by a single function call. By default the model's objective function is optimized. The function returns a solution object, in which the objective value and the fluxes are stored in `solution.objective_value` and `solution.fluxes`.

In [2]:
solution = sd.fba(model)

print(f"Maximum growth: {solution.objective_value}.")

Maximum growth: 0.8739215069684305.


You may also use a custom objective and change the optimization sense. Here we minimize the Glucose uptake rate through the PTS:

In [3]:
solution = sd.fba(model,obj='GLCpts',obj_sense='minimize')

print(f"Minimum flux through GLCpts: {solution.objective_value}.")

Minimum flux through GLCpts: 0.4794285714285715.


We can also consider custom constraints (in this case limited oxygen uptake and an increased fixed ATP maintenance demand):

In [4]:
solution = sd.fba(model,constraints=['-EX_o2_e <= 5', 'ATPM = 20'])

print(f"Maximum growth at limited oxygen uptake and high ATP maintenance: {solution.objective_value}.")

Maximum growth at limited oxygen uptake and high ATP maintenance: 0.26305573292588313.


##### Parsimonious FBA (pFBA)

Parsimonious flux balance analysis optimizes a flux rate under the given steady-state network constraints, but also minimizes the sum of absolute fluxes to achieve this optimum. One can write:
$$\text{minimize}\:\:\:\Sigma | v_i | \\ \text{subject to} \\$$ 
$$\text{maximize}\:\:\: v_{growth} \\ \text{subject to} \\ \mathbf{S\, v = 0} \\ \mathbf{lb \le v \le ub}$$

pFBA is the simpleset way of emulating the cell's enzyme cost minimization (after an assumed growth maximization). pFBA solutions are more often "unique" than pure FBA solutions, since the outer minimization leaves fewer degrees of freedom in the solution space. StrainDesign computes pFBA solutions when you pass the 'pFBA'-argument with a value of 1. 

In [30]:
sd.fba(model,pfba=1)

,fluxes,reduced_costs
ACALD,0.000000,None
ACALDt,0.000000,None
ACKr,0.000000,None
ACONTa,6.007250,None
ACONTb,6.007250,None
...,...,...
TALA,1.496984,None
THD2,0.000000,None
TKT1,1.496984,None
TKT2,1.181498,None


## Solver selection

For any type of LP or MILP optimization, 4 different sovers are supported: **GLPK** (which is built into COBRApy/optlang), **IBM CPLEX**, **Gurobi** and the open source solver **SCIP**. You can query the available solvers by accessing the set at straindesign.avail_solvers.

In [10]:
sd.avail_solvers

['glpk', 'cplex', 'gurobi', 'scip']

You may enforce the use of a specific solver by specifying the "solver"-keyword. To enforce the use of GLPK, use: 

In [11]:
solution = sd.fba(model, solver='cplex')

print(f"Maximum growth: {solution.objective_value}.")

Maximum growth: 0.8739215069684826.


By default, the automatic solver selection uses COBRApy's selection. Therefore, StrainDesign will try to use the model's selected solver:

In [25]:
print(f"When the model\'s solver is \'{model.solver.configuration}', StrainDesign selects {sd.select_solver(None,model)}.")

When the model's solver is '<optlang.gurobi_interface.Configuration object at 0x000001C9B8A5A970>', StrainDesign selects gurobi.


Otherwise COBRApy's global configuration is used.

In [28]:
print(f"COBRApy\'s solver is \'{cobra.Configuration().solver.__name__}', StrainDesign selects {sd.select_solver()}.")

COBRApy's solver is 'optlang.gurobi_interface', StrainDesign selects gurobi.


## Flux ranges (FVA)

In [ ]:
solution = sd.fva(model)
print(solution)

## Yield optimization